In [33]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as integrate
import scipy
import matplotlib.patches as mpat

In [1]:
def Plottit(data,mu,t_Diagramm = 0, L = 1025):
    
    Sig_Diag = data[:L]
    Sig_OffDiag = data[L:]
    h_vals = -np.ones(L-1)

    
    Diag = np.real(Sig_Diag) 
    OffDiag = np.real(Sig_OffDiag + h_vals)
    
    A = scipy.linalg.eigh_tridiagonal(Diag,OffDiag)

    Eigenvals,Eigenvecs = A
    epsilon = Eigenvals-mu
    print(max(np.imag(Eigenvecs).flatten())) # just to crosscheck that nothing is imaginary
    #Sorting the eigenvals and eigenvecs...
    Indices = np.argsort(epsilon)
    epsilon = np.real(epsilon[Indices])
    Eigenvecs = Eigenvecs[:,Indices]

    
    epsilon_avg = (epsilon[1::2]+epsilon[:-1:2])/2

    deriv_epsilon = 2./(epsilon_avg[1:]-epsilon_avg[:-1])
    epsilon_avg = (epsilon_avg[1:]+epsilon_avg[:-1])/2
    
    #j_0 = 0
    j_0 = 0

    omegas = np.abs(Eigenvecs[j_0,:])**2
    #now averaging:
    vals = []
    for i in range(0,len(omegas),2):
        if((i+1)<=(L-1)):
            vals.append((omegas[i]+omegas[i+1])/2)
    vals = np.array(vals)

    vals = (vals[1:]+vals[:-1])/2

    Plotpoints = vals * deriv_epsilon
    plt.plot(epsilon_avg,Plotpoints,"r.")
    plt.xlim(10**-4,1)
    plt.xscale("log")
    plt.yscale("log")
    #plt.show()
    return epsilon_avg,Plotpoints


In [2]:
def Plotit_with_k(data,mu,t,L = 2000,Quickreturn = False,QuickreturnData = False,Saveit = False,Eps_precalced = [],Pltpts_precalcd = []):
    
    #This function returns the LDOS and the omega values at which it was calculated for a given Sigma. 
    #data is the Self-energy in the form of a (N+ N-1) long array containg first its diagonal then its off-diagonal elements.
    #mu is the chem.pot. at which to calculate the LDOS,
    #t denotes the cross-chain hopping, which is called t' in the thesis.
    #L corresponds to the system size N
    #If Quickreturn ==True, the function will only return rho(0) and not calculate the LDOS for different points
    
    
    if(len(Eps_precalced)==0):
        epsilon_avg,Plotpoints = Plottit(data,mu,t,L)
    else:
        epsilon_avg = Eps_precalced
        Plotpoints = Pltpts_precalcd
    def LDOS(omega):
        if(omega<epsilon_avg[0] or omega>epsilon_avg[-1]):
            return 0
        for i in range(len(epsilon_avg)-1):
            if(epsilon_avg[i]<=omega and epsilon_avg[i+1]>omega):
                return Plotpoints[i] + (Plotpoints[i+1]-Plotpoints[i])*(omega-epsilon_avg[i])/(epsilon_avg[i+1]-epsilon_avg[i])

    def LDOS_integrand(k,omega,t):

        return LDOS(omega-2*t*np.cos(k))
    def New_LDOS(omega,t):
        return 1/np.pi * integrate.quad(LDOS_integrand,0,np.pi,args = (omega,t))[0]
    if(QuickreturnData == True):
        return epsilon_avg,Plotpoints
    if(Quickreturn == True):
        return New_LDOS(0,t)
    omegars = np.logspace(-3,1,400)
    LDOSlist = []
    for omegar in omegars:
        LDOSlist.append(New_LDOS(omegar,t))
    #plt.plot(epsilon_avg,Plotpoints,"r.")

    plt.plot(Epsilon_N8000_U1[0],Plotpoints_N8000_U1[0],"b.")
    plt.plot(omegars,LDOSlist,"red",linestyle = "--")

    plt.xlim(10**-4,1)
    plt.ylim(10**-2,0.3)
    plt.xscale("log")
    plt.yscale("log")
    #plt.text(4,0.2,"{}".format(New_LDOS(0,t)))
    plt.text(4,0.15,r"$t^\prime$ = {}".format(t))
    #plt.ylim(-0.02,0.28)
    
    plt.xlabel("$\omega$",fontsize = 14)
    if(Saveit == True):
        plt.savefig("LDOS_Boundary_t{}.png".format(t),dpi = 600)

    plt.show()
    return np.array(LDOSlist),np.array(omegars)